In [1]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Importer les bibliothèques
import os
import duckdb
import subprocess
import pandas as pd

In [3]:
# Fonction pour lister les fichiers dans HDFS
def list_hdfs_files(path):
    print(f"Listing HDFS files in path: {path}")
    
    # Exécuter la commande HDFS et capturer la sortie
    result = subprocess.run(['hdfs', 'dfs', '-ls', path], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    
    # Vérifier s'il y a une erreur
    if result.returncode != 0:
        print(f"Erreur lors de l'exécution de la commande HDFS: {result.stderr}")
        return []

    print("Commande HDFS exécutée avec succès.")
    
    files = []
    # Parcourir la sortie de la commande pour trouver les fichiers CSV correspondants
    for line in result.stdout.split('\n'):
        print(f"Analyzing line: {line}")  # Debug pour voir chaque ligne retournée
        if "On_Time_Marketing_Carrier_On_Time_Performance" in line and line.endswith(".csv"):
            file_path = line.split()[-1]  # Récupérer le chemin complet du fichier
            files.append(file_path)
            print(f"File found: {file_path}")  # Debug pour voir chaque fichier trouvé

    return files

In [4]:
# Lister les fichiers dans /data
hdfs_path = "/data"
files = list_hdfs_files(hdfs_path)

print(f"Fichiers trouvés : {files}")

Listing HDFS files in path: /data
Commande HDFS exécutée avec succès.
Analyzing line: Found 67 items
Analyzing line: drwxr-xr-x   - root supergroup          0 2024-10-19 21:14 /data/2019_01
Analyzing line: drwxr-xr-x   - root supergroup          0 2024-10-19 21:14 /data/2019_02
Analyzing line: drwxr-xr-x   - root supergroup          0 2024-10-19 21:14 /data/2019_03
Analyzing line: drwxr-xr-x   - root supergroup          0 2024-10-19 21:14 /data/2019_04
Analyzing line: drwxr-xr-x   - root supergroup          0 2024-10-19 21:14 /data/2019_05
Analyzing line: drwxr-xr-x   - root supergroup          0 2024-10-19 21:14 /data/2019_06
Analyzing line: drwxr-xr-x   - root supergroup          0 2024-10-19 21:14 /data/2019_07
Analyzing line: drwxr-xr-x   - root supergroup          0 2024-10-19 21:14 /data/2019_08
Analyzing line: drwxr-xr-x   - root supergroup          0 2024-10-19 21:14 /data/2019_09
Analyzing line: drwxr-xr-x   - root supergroup          0 2024-10-19 21:14 /data/2019_10
Analyzing

In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql.types import TimestampType
from pyspark.sql.functions import udf
from pyspark.sql.window import Window
from pyspark.sql.types import *
from pyspark.sql.functions import to_timestamp, col

In [6]:
    # Créer une session Spark
    spark = SparkSession.builder \
        .appName("FlightDataAnalysis") \
        .config("spark.hadoop.fs.defaultFS", "hdfs://hadoop-namenode:9000") \
        .getOrCreate()
    
    # Vérifier que la session Spark fonctionne bien
    spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [7]:
# Définir le chemin des fichiers CSV dans HDFS
path = "hdfs://hadoop-namenode:9000/data/2019_01/On_Time_Marketing_Carrier_On_Time_Performance*.csv"

# Lire les fichiers CSV avec Spark
flight_data_df = spark.read.format("csv").option("header", "true").load(path, inferSchema=True)

# Afficher le schéma des données
flight_data_df.printSchema()

# Afficher les premières lignes
flight_data_df.show(5)

root
 |-- Year: integer (nullable = true)
 |-- Quarter: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- FlightDate: date (nullable = true)
 |-- Marketing_Airline_Network: string (nullable = true)
 |-- Operated_or_Branded_Code_Share_Partners: string (nullable = true)
 |-- DOT_ID_Marketing_Airline: integer (nullable = true)
 |-- IATA_Code_Marketing_Airline: string (nullable = true)
 |-- Flight_Number_Marketing_Airline: integer (nullable = true)
 |-- Originally_Scheduled_Code_Share_Airline: string (nullable = true)
 |-- DOT_ID_Originally_Scheduled_Code_Share_Airline: integer (nullable = true)
 |-- IATA_Code_Originally_Scheduled_Code_Share_Airline: string (nullable = true)
 |-- Flight_Num_Originally_Scheduled_Code_Share_Airline: integer (nullable = true)
 |-- Operating_Airline : string (nullable = true)
 |-- DOT_ID_Operating_Airline: integer (nullable = true)
 |-- IATA_Code_Operating_A

24/10/20 00:24:24 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
24/10/20 00:24:25 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Year, Quarter, Month, DayofMonth, DayOfWeek, FlightDate, Marketing_Airline_Network, Operated_or_Branded_Code_Share_Partners, DOT_ID_Marketing_Airline, IATA_Code_Marketing_Airline, Flight_Number_Marketing_Airline, Originally_Scheduled_Code_Share_Airline, DOT_ID_Originally_Scheduled_Code_Share_Airline, IATA_Code_Originally_Scheduled_Code_Share_Airline, Flight_Num_Originally_Scheduled_Code_Share_Airline, Operating_Airline , DOT_ID_Operating_Airline, IATA_Code_Operating_Airline, Tail_Number, Flight_Number_Operating_Airline, OriginAirportID, OriginAirportSeqID, OriginCityMarketID, Origin, OriginCityName, OriginState, OriginStateFips, OriginStateName, OriginWac, DestAirportID, DestAirportSeqID, DestCityMarketID, Dest, DestC

+----+-------+-----+----------+---------+----------+-------------------------+---------------------------------------+------------------------+---------------------------+-------------------------------+---------------------------------------+----------------------------------------------+-------------------------------------------------+--------------------------------------------------+------------------+------------------------+---------------------------+-----------+-------------------------------+---------------+------------------+------------------+------+-------------------+-----------+---------------+---------------+---------+-------------+----------------+----------------+----+-----------------+---------+-------------+-------------+-------+----------+-------+--------+---------------+--------+--------------------+----------+-------+---------+--------+------+----------+-------+--------+---------------+--------+------------------+----------+---------+----------------+--------+---

In [8]:
# Check for duplicates and remove them
cleaned_df = flight_data_df.dropDuplicates()

# # Check for null values and drop rows with any null values
# cleaned_df = cleaned_df.dropna()

# Verify the cleaned data
cleaned_df.show(5)

24/10/20 00:24:26 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Year, Quarter, Month, DayofMonth, DayOfWeek, FlightDate, Marketing_Airline_Network, Operated_or_Branded_Code_Share_Partners, DOT_ID_Marketing_Airline, IATA_Code_Marketing_Airline, Flight_Number_Marketing_Airline, Originally_Scheduled_Code_Share_Airline, DOT_ID_Originally_Scheduled_Code_Share_Airline, IATA_Code_Originally_Scheduled_Code_Share_Airline, Flight_Num_Originally_Scheduled_Code_Share_Airline, Operating_Airline , DOT_ID_Operating_Airline, IATA_Code_Operating_Airline, Tail_Number, Flight_Number_Operating_Airline, OriginAirportID, OriginAirportSeqID, OriginCityMarketID, Origin, OriginCityName, OriginState, OriginStateFips, OriginStateName, OriginWac, DestAirportID, DestAirportSeqID, DestCityMarketID, Dest, DestCityName, DestState, DestStateFips, DestStateName, DestWac, CRSDepTime, DepTime, DepDelay, DepDelayMinutes, DepDel15, DepartureDelayGroups, DepTimeBlk, TaxiOut, WheelsOff, WheelsOn,

+----+-------+-----+----------+---------+----------+-------------------------+---------------------------------------+------------------------+---------------------------+-------------------------------+---------------------------------------+----------------------------------------------+-------------------------------------------------+--------------------------------------------------+------------------+------------------------+---------------------------+-----------+-------------------------------+---------------+------------------+------------------+------+--------------------+-----------+---------------+---------------+---------+-------------+----------------+----------------+----+-------------------+---------+-------------+--------------+-------+----------+-------+--------+---------------+--------+--------------------+----------+-------+---------+--------+------+----------+-------+--------+---------------+--------+------------------+----------+---------+----------------+--------

In [9]:
# Define the columns to keep
columns_to_keep = [
    "Year", "Month", "DayofMonth", "DayOfWeek", "FlightDate",
    "Marketing_Airline_Network", "Flight_Number_Marketing_Airline",
    "Origin", "Dest", "DepDelay", "ArrDelay", "Cancelled",
    "CarrierDelay", "WeatherDelay", "NASDelay", "SecurityDelay", "LateAircraftDelay"
]

# Select the relevant columns from the cleaned DataFrame
filtered_df = flight_data_df.select(*columns_to_keep)

filtered_df.show(5)

+----+-----+----------+---------+----------+-------------------------+-------------------------------+------+----+--------+--------+---------+------------+------------+--------+-------------+-----------------+
|Year|Month|DayofMonth|DayOfWeek|FlightDate|Marketing_Airline_Network|Flight_Number_Marketing_Airline|Origin|Dest|DepDelay|ArrDelay|Cancelled|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+----+-----+----------+---------+----------+-------------------------+-------------------------------+------+----+--------+--------+---------+------------+------------+--------+-------------+-----------------+
|2019|    1|         4|        5|2019-01-04|                       UA|                           5657|   SBP| SFO|    -7.0|   -26.0|      0.0|        null|        null|    null|         null|             null|
|2019|    1|         4|        5|2019-01-04|                       UA|                           5658|   IAH| XNA|    -5.0|     1.0|      0.0|        null|     

In [10]:
import duckdb
import pandas as pd

# Create a connection to DuckDB
con = duckdb.connect(database=':memory:')

# Step 4: Convert Spark DataFrame to Pandas DataFrame
pandas_df = filtered_df.toPandas()

In [11]:
# Create an empty table in DuckDB
con.execute("CREATE TABLE IF NOT EXISTS flight_data AS SELECT * FROM pandas_df")


In [12]:
# Optional: Verify the data insertion
result = con.execute("SELECT * FROM flight_data LIMIT 5").fetchall()
print(result)

[(2019, 1, 4, 5, datetime.date(2019, 1, 4), 'UA', 5657, 'SBP', 'SFO', -7.0, -26.0, 0.0, None, None, None, None, None), (2019, 1, 4, 5, datetime.date(2019, 1, 4), 'UA', 5658, 'IAH', 'XNA', -5.0, 1.0, 0.0, None, None, None, None, None), (2019, 1, 4, 5, datetime.date(2019, 1, 4), 'UA', 5658, 'SGF', 'IAH', -6.0, -17.0, 0.0, None, None, None, None, None), (2019, 1, 4, 5, datetime.date(2019, 1, 4), 'UA', 5659, 'ISN', 'DEN', -21.0, -29.0, 0.0, None, None, None, None, None), (2019, 1, 4, 5, datetime.date(2019, 1, 4), 'UA', 5660, 'MKE', 'DEN', -10.0, -31.0, 0.0, None, None, None, None, None)]


In [15]:
destination_path = "hdfs://hadoop-namenode:9000/done/"

# Step 3: List all files matching the pattern in the source directory
list_files_command = ["hdfs", "dfs", "-ls", path]
result = subprocess.run(list_files_command, capture_output=True, text=True)
files = [line.split()[-1] for line in result.stdout.splitlines() if line.startswith('-')]

# Step 4: Process each file
for file_path in files:
    print(f"Processing file: {file_path}")

    # Optional: Verify the data insertion
    result = con.execute("SELECT * FROM flight_data LIMIT 5").fetchall()
    print(result)

    # Construct the dynamic destination path
    file_name = os.path.basename(file_path)

    # Create the destination directory if it doesn't exist
    mkdir_command = ["hdfs", "dfs", "-mkdir", "-p", destination_path]
    subprocess.run(mkdir_command)
    
    # Move the processed file to the destination directory
    move_command = ["hdfs", "dfs", "-mv", file_path, destination_path]
    subprocess.run(move_command)

    print(f"Moved file: {file_path} to {destination_path}")



Processing file: hdfs://hadoop-namenode:9000/data/2019_01/On_Time_Marketing_Carrier_On_Time_Performance_Beginning_January_2018_2019_1.csv
[(2019, 1, 4, 5, datetime.date(2019, 1, 4), 'UA', 5657, 'SBP', 'SFO', -7.0, -26.0, 0.0, None, None, None, None, None), (2019, 1, 4, 5, datetime.date(2019, 1, 4), 'UA', 5658, 'IAH', 'XNA', -5.0, 1.0, 0.0, None, None, None, None, None), (2019, 1, 4, 5, datetime.date(2019, 1, 4), 'UA', 5658, 'SGF', 'IAH', -6.0, -17.0, 0.0, None, None, None, None, None), (2019, 1, 4, 5, datetime.date(2019, 1, 4), 'UA', 5659, 'ISN', 'DEN', -21.0, -29.0, 0.0, None, None, None, None, None), (2019, 1, 4, 5, datetime.date(2019, 1, 4), 'UA', 5660, 'MKE', 'DEN', -10.0, -31.0, 0.0, None, None, None, None, None)]
Moved file: hdfs://hadoop-namenode:9000/data/2019_01/On_Time_Marketing_Carrier_On_Time_Performance_Beginning_January_2018_2019_1.csv to hdfs://hadoop-namenode:9000/done/
